In [2]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Activation
import keras
from keras.optimizers import SGD
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

2024-05-28 07:38:18.354942: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 07:38:18.414797: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 07:38:18.814225: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 07:38:20.541104: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# 데이터프레임 읽기
df = pd.read_csv('./weather_kor4.csv')

In [4]:
df.columns

Index(['Unnamed: 0', '날짜', '기온', '강수량', '풍속_num', '풍향_num', '습도', '현지기압',
       '전운량', 'aplYmd', '날씨코드', 'wetrTxt', 'wetrSplit', '현상번호_split',
       'weather_1', 'weather_2', '풍속', '풍향', '일조량', '시간당 강수량', 'timestamp',
       'W1', 'W2'],
      dtype='object')

In [42]:
df 

,Unnamed: 0,날짜,기온,강수량,풍속_num,풍향_num,습도,현지기압,전운량,aplYmd,...,현상번호_split,weather_1,weather_2,풍속,풍향,일조량,시간당 강수량,timestamp,W1,W2
0,0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,NE,-1.0,없음,2019-12-29 00:00:00,1,1
1,1,2019-12-29 01:00,1.2,0.0,1.8,70.0,54,1017.9,6.0,20191229,...,"['92', '01']",구름많음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 01:00:00,0,1
2,2,2019-12-29 02:00,1.3,0.0,2.4,70.0,53,1017.9,4.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 02:00:00,1,1
3,3,2019-12-29 03:00,1.4,0.0,1.8,70.0,52,1017.9,5.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 03:00:00,1,1
4,4,2019-12-29 04:00,1.4,0.0,2.6,70.0,51,1017.1,5.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 04:00:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37690,37690,2024-05-23 19:00,22.5,0.0,3.5,270.0,55,999.5,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,W,1.0,없음,2024-05-23 19:00:00,1,1
37691,37691,2024-05-23 20:00,20.5,0.0,3.0,290.0,68,1000.3,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,0.0,없음,2024-05-23 20:00:00,1,1
37692,37692,2024-05-23 21:00,18.6,0.0,2.8,270.0,73,1001.0,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,W,-1.0,없음,2024-05-23 21:00:00,1,1
37693,37693,2024-05-23 22:00,17.7,0.0,2.9,290.0,79,1001.6,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,-1.0,없음,2024-05-23 22:00:00,1,1


In [35]:
df.isna().sum()

Unnamed: 0    0
날짜            0
기온            0
강수량           0
풍속_num        0
풍향_num        0
습도            0
현지기압          0
전운량           0
aplYmd        0
날씨코드          0
wetrTxt       0
wetrSplit     0
현상번호_split    0
weather_1     0
weather_2     0
풍속            0
풍향            0
일조량           0
시간당 강수량       0
timestamp     0
W1            0
W2            0
dtype: int64

In [8]:
df.drop(columns={'W1','W2'},inplace=True)

In [9]:
df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
df["시간당 강수량"] = df["시간당 강수량"].fillna("없음")
# 데이터프레임 nan값 지우기
df = df.dropna()

In [37]:
df

,Unnamed: 0,날짜,기온,강수량,풍속_num,풍향_num,습도,현지기압,전운량,aplYmd,...,현상번호_split,weather_1,weather_2,풍속,풍향,일조량,시간당 강수량,timestamp,W1,W2
0,0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,NE,-1.0,없음,2019-12-29 00:00:00,1,1
1,1,2019-12-29 01:00,1.2,0.0,1.8,70.0,54,1017.9,6.0,20191229,...,"['92', '01']",구름많음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 01:00:00,0,1
2,2,2019-12-29 02:00,1.3,0.0,2.4,70.0,53,1017.9,4.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 02:00:00,1,1
3,3,2019-12-29 03:00,1.4,0.0,1.8,70.0,52,1017.9,5.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 03:00:00,1,1
4,4,2019-12-29 04:00,1.4,0.0,2.6,70.0,51,1017.1,5.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 04:00:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37690,37690,2024-05-23 19:00,22.5,0.0,3.5,270.0,55,999.5,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,W,1.0,없음,2024-05-23 19:00:00,1,1
37691,37691,2024-05-23 20:00,20.5,0.0,3.0,290.0,68,1000.3,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,0.0,없음,2024-05-23 20:00:00,1,1
37692,37692,2024-05-23 21:00,18.6,0.0,2.8,270.0,73,1001.0,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,W,-1.0,없음,2024-05-23 21:00:00,1,1
37693,37693,2024-05-23 22:00,17.7,0.0,2.9,290.0,79,1001.6,0.0,20240523,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,-1.0,없음,2024-05-23 22:00:00,1,1


In [10]:
# 인덱스제거
df.reset_index(drop=True,inplace=True)
# weather_1에 라벨인코딩
encoder = LabelEncoder()
encoder.fit(df["weather_1"])
encoded_data = encoder.transform(df["weather_1"])
# # weather_2에 라벨인코딩
encoder1 = LabelEncoder()
encoder1.fit(df["weather_2"])
encoded_data1 = encoder1.transform(df["weather_2"])
# data_Frame으로 변환
encoded_data_df = pd.DataFrame(encoded_data,columns=['W1'])
# data_Frame으로 변환
encoded_data_df1 = pd.DataFrame(encoded_data1,columns=['W2'])

In [11]:
from datetime import datetime
df['timestamp'] = pd.to_datetime(df['날짜'])
# data_frame 통합
df = pd.concat([df,encoded_data_df,encoded_data_df1],axis=1)

In [12]:
input_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']
output_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']

In [13]:
df

,날짜,기온,강수량,풍속_num,풍향_num,습도,현지기압,전운량,aplYmd,날씨코드,...,현상번호_split,weather_1,weather_2,풍속,풍향,일조량,시간당 강수량,timestamp,W1,W2
0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,NE,-1.0,없음,2019-12-29 00:00:00,1,1
1,2019-12-29 01:00,1.2,0.0,1.8,70.0,54,1017.9,6.0,20191229,9.0,...,"['92', '01']",구름많음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 01:00:00,0,1
2,2019-12-29 02:00,1.3,0.0,2.4,70.0,53,1017.9,4.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 02:00:00,1,1
3,2019-12-29 03:00,1.4,0.0,1.8,70.0,52,1017.9,5.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 03:00:00,1,1
4,2019-12-29 04:00,1.4,0.0,2.6,70.0,51,1017.1,5.0,20191229,9.0,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,ENE,-1.0,없음,2019-12-29 04:00:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37690,2024-05-23 19:00,22.5,0.0,3.5,270.0,55,999.5,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,W,1.0,없음,2024-05-23 19:00:00,1,1
37691,2024-05-23 20:00,20.5,0.0,3.0,290.0,68,1000.3,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,0.0,없음,2024-05-23 20:00:00,1,1
37692,2024-05-23 21:00,18.6,0.0,2.8,270.0,73,1001.0,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,W,-1.0,없음,2024-05-23 21:00:00,1,1
37693,2024-05-23 22:00,17.7,0.0,2.9,290.0,79,1001.6,0.0,20240523,17.0,...,['90'],맑음,둘다아님,약한 강한 바람,WNW,-1.0,없음,2024-05-23 22:00:00,1,1


In [14]:
df.columns

Index(['날짜', '기온', '강수량', '풍속_num', '풍향_num', '습도', '현지기압', '전운량', 'aplYmd',
       '날씨코드', 'wetrTxt', 'wetrSplit', '현상번호_split', 'weather_1', 'weather_2',
       '풍속', '풍향', '일조량', '시간당 강수량', 'timestamp', 'W1', 'W2'],
      dtype='object')

In [15]:
# 데이터 전처리 변수마다 범위가 다르므로 다르게 적용
from sklearn.preprocessing import MinMaxScaler
scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    df[feature] = scaler.fit_transform(df[[feature]])
    # print(scaled_data)
    scalers[feature] = scaler
# 시퀀스 데이터 생성
sequence_length = 720  # 예측에 사용할 시퀀스 길이
X1 = []
Y1 = []
for i in tqdm(range(len(df) - sequence_length)):
    # 각 변수에 해당하는 데이터를 스케일링하여 저장
    scaled_features = []
    for feature in input_features:
        scaled_feature = scalers[feature].transform(df[[feature]].iloc[i:i+sequence_length])
        scaled_features.append(scaled_feature)
    # 각 변수를 열로 가지는 2차원 배열로 변환하여 X에 추가
    X1.append(np.concatenate(scaled_features, axis=1))
    # 출력 변수는 스케일링할 필요가 없으므로 그대로 추가
    Y1.append(df[output_features].iloc[i+sequence_length])
X2 = np.array(X1)
Y2 = np.array(Y1)


100%|█████████████████████████████████████████████████████████████████████████████| 36975/36975 [21:35<00:00, 28.55it/s]


In [16]:
train_length = int(len(X2)*0.8)
x_train = X2[0:train_length]
y_train = Y2[0:train_length]

In [17]:
y_train.shape

(29580, 9)

In [18]:
x_train.shape

(29580, 720, 9)

In [19]:
x_test = X2[train_length:]
y_test = Y2[train_length:]

In [20]:
x_test.shape

(7395, 720, 9)

In [21]:
valid_length = int(len(x_train)*0.8)
valid_length

23664

In [22]:
new_x_train = x_train[0:valid_length]
new_y_train = y_train[0:valid_length]

In [23]:
x_val = x_train[valid_length:]
y_val = y_train[valid_length:]

In [24]:
x_train.shape

(29580, 720, 9)

In [25]:
x_val.shape

(5916, 720, 9)

In [26]:
x_test.shape

(7395, 720, 9)

In [27]:
import tensorflow as tf

In [28]:
# LSTM 모델 구성
# 50은 뉴런의 수 조절해야하는 파라미터 과적합 조심!
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(output_features))
])

/home/encore/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
# 모델 컴파일
model.compile(loss='mse', optimizer='adam')

In [30]:
# 모델 조기종료
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=20,        
    restore_best_weights=True  
)

In [31]:
x_train.shape

(29580, 720, 9)

In [32]:
# 모델 학습
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping_callback]  
)

Epoch 1/5


2024-05-28 03:09:45.394483: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 766713600 exceeds 10% of free system memory.


323/925 ━━━━━━━━━━━━━━━━━━━━ 5:30 549ms/step - loss: 0.1085


KeyboardInterrupt



In [ ]:
# 모델 평가
loss = model.evaluate(x_test, y_test)
print("Test Loss:", loss)

In [ ]:
last_date = df['timestamp'].iloc[-1]  
next_date = last_date + timedelta(days=1) 
print("다음 날짜:", next_date)

In [ ]:
last_sequence = df[input_features].iloc[-sequence_length:, :]
next_day_prediction = model.predict(np.array([last_sequence]))
np.set_printoptions(suppress=True)
for feature, scaler in scalers.items():
    if feature in output_features:
        next_day_prediction[0, output_features.index(feature)] = scaler.inverse_transform(next_day_prediction[0, output_features.index(feature)].reshape(-1, 1)).flatten()
print("다음 날 기상 예측:", next_day_prediction)


In [ ]:
# model.save("lstm_model.h5")

In [ ]:
# 다음 날의 기상 예측
last_sequence = df[input_features].iloc[-sequence_length:, :] 
next_day_predictions = []
for i in tqdm(range(72)):
    next_day_prediction = model.predict(np.array([last_sequence]))
    for feature, scaler in scalers.items():
        if feature in output_features:
            next_day_prediction[0, output_features.index(feature)] = scaler.inverse_transform(next_day_prediction[0, output_features.index(feature)].reshape(-1, 1)).flatten()
    next_day_predictions.append(next_day_prediction.flatten())
    last_sequence = np.vstack([last_sequence[1:], next_day_prediction])
print("72시간 동안의 기상 예측:")
for i, prediction in enumerate(next_day_predictions, start=1):
    print(f"시간 {i}: {prediction}")


In [ ]:
from sklearn.metrics import accuracy_score
# 모델의 예측 생성
y_pred = model.predict(x_test)
# 모델의 예측값을 클래스로 변환 (예: 0 또는 1)
y_pred_class = np.argmax(y_pred, axis=1)

# 정확도 계산
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_class)
print("Accuracy:", accuracy)

In [ ]:
next_day_predictions

In [ ]:
next_day_prediction

In [ ]:
next_day_predictions[0]

In [ ]:
# model.save('lstm_model.h5')
model.save('lstm_model.keras')

In [ ]:
model.save('lstm_model.h5')